In [ ]:
import pandas as pd
data = pd.read_csv("Tweets_Data.csv",encoding = "ISO-8859-1")

In [ ]:
data.head()

,Unnamed: 0,Text
0,1,Today is @ great day
1,2,Today is bad day
2,3,Having fun???
3,4,Its too difficult
4,5,Good morning


In [ ]:
data = data.iloc[:,1:]
data.head()

,Text
0,Today is @ great day
1,Today is bad day
2,Having fun???
3,Its too difficult
4,Good morning


In [ ]:
data.isnull().sum()

Text    0
dtype: int64

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    9 non-null      object
dtypes: object(1)
memory usage: 200.0+ bytes


In [ ]:
import re #regular expression
import string

def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub("[0-9" "]+"," ",text)
    text = re.sub('[‘’“”…]', '', text)
    return text

clean = lambda x: clean_text(x)
data['Text'] = data.Text.apply(clean)
data.Text
data_df = pd.DataFrame(data)
data_df

,Text
0,today is great day
1,today is bad day
2,having fun
3,its too difficult
4,good morning
5,its worth it
6,this is not an easy task
7,bad day
8,this is kinda boring


In [ ]:
#Word frequency
freq = pd.Series(' '.join(data['Text']).split()).value_counts()[:20] # for top 20
freq

is           4
day          3
today        2
bad          2
its          2
this         2
it           1
kinda        1
task         1
easy         1
an           1
not          1
good         1
worth        1
morning      1
difficult    1
too          1
fun          1
having       1
great        1
dtype: int64

In [ ]:
#removing stopwords
from nltk.corpus import stopwords
stop = pd.read_csv('stop.txt')
data['Text'] = data['Text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [ ]:
#word frequency after removal of stopwords
freq_Sw = pd.Series(' '.join(data['Text']).split()).value_counts()[:20] # for top 20
freq_Sw

is           4
day          3
today        2
bad          2
its          2
this         2
it           1
kinda        1
task         1
easy         1
an           1
not          1
good         1
worth        1
morning      1
difficult    1
too          1
fun          1
having       1
great        1
dtype: int64

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# count vectoriser tells the frequency of a word.
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
vectorizer = CountVectorizer(min_df = 1, max_df = 0.9)
X = vectorizer.fit_transform(data["Text"])
word_freq_df = pd.DataFrame({'term': vectorizer.get_feature_names(), 'occurrences':np.asarray(X.sum(axis=0)).ravel().tolist()})
word_freq_df['frequency'] = word_freq_df['occurrences']/np.sum(word_freq_df['occurrences'])
#print(word_freq_df.sort('occurrences',ascending = False).head())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
word_freq_df.head(30)

,term,occurrences,frequency
0,an,1,0.033333
1,bad,2,0.066667
2,boring,1,0.033333
3,day,3,0.100000
4,difficult,1,0.033333
5,easy,1,0.033333
6,fun,1,0.033333
7,good,1,0.033333
8,great,1,0.033333
9,having,1,0.033333


In [ ]:
#TFIDF - Term frequency inverse Document Frequency
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000, max_df = 0.5, smooth_idf=True) #keep top 1000 words
doc_vec = vectorizer.fit_transform(data["Text"])
names_features = vectorizer.get_feature_names()
dense = doc_vec.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns = names_features)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
df

,bad,boring,day,difficult,easy,fun,good,great,having,kinda,morning,task,today,worth
0,0.000000,0.000000,0.489289,0.0,0.000000,0.000000,0.000000,0.666271,0.000000,0.000000,0.000000,0.000000,0.562743,0.0
1,0.602368,0.000000,0.523742,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.602368,0.0
2,0.000000,0.000000,0.000000,0.0,0.000000,0.707107,0.000000,0.000000,0.707107,0.000000,0.000000,0.000000,0.000000,0.0
3,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.707107,0.000000,0.000000,0.000000,0.707107,0.000000,0.000000,0.0
5,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
6,0.000000,0.000000,0.000000,0.0,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000,0.0
7,0.754641,0.000000,0.656138,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
8,0.000000,0.707107,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000,0.000000,0.000000,0.0


In [ ]:
#Bi-gram
def get_top_n2_words(corpus, n=None):
    vec1 = CountVectorizer(ngram_range=(2,2),  #for tri-gram, put ngram_range=(3,3)
            max_features=2000).fit(corpus)
    bag_of_words = vec1.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in
                  vec1.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1],
                reverse=True)
    return words_freq[:n]

In [ ]:
top2_words = get_top_n2_words(data["Text"], n=200) #top 200
top2_df = pd.DataFrame(top2_words)
top2_df.columns=["Bi-gram", "Freq"]
top2_df.head()

,Bi-gram,Freq
0,today is,2
1,bad day,2
2,this is,2
3,is great,1
4,great day,1


In [ ]:
top2_df

,Bi-gram,Freq
0,today is,2
1,bad day,2
2,this is,2
3,is great,1
4,great day,1
5,is bad,1
6,having fun,1
7,its too,1
8,too difficult,1
9,good morning,1


In [ ]:
#Tri-gram
def get_top_n3_words(corpus, n=None):
    vec1 = CountVectorizer(ngram_range=(3,3),
           max_features=2000).fit(corpus)
    bag_of_words = vec1.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in
                  vec1.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1],
                reverse=True)
    return words_freq[:n]
top3_words = get_top_n3_words(data["Text"], n=200)
top3_df = pd.DataFrame(top3_words)
top3_df.columns=["Tri-gram", "Freq"]

In [ ]:
top3_df

,Tri-gram,Freq
0,today is great,1
1,is great day,1
2,today is bad,1
3,is bad day,1
4,its too difficult,1
5,its worth it,1
6,this is not,1
7,is not an,1
8,not an easy,1
9,an easy task,1


In [ ]:
#Sentiment analysis
afinn = pd.read_csv('Afinn.csv', sep=',', encoding='latin-1')
afinn.shape
(2477, 2)
afinn.head()

,word,value
0,abandon,-2
1,abandoned,-2
2,abandons,-2
3,abducted,-2
4,abduction,-2


In [ ]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))
affinity_scores = afinn.set_index('word')['value'].to_dict()
take(20, affinity_scores.items())

[('abandon', -2),
 ('abandoned', -2),
 ('abandons', -2),
 ('abducted', -2),
 ('abduction', -2),
 ('abductions', -2),
 ('abhor', -3),
 ('abhorred', -3),
 ('abhorrent', -3),
 ('abhors', -3),
 ('abilities', 2),
 ('ability', 2),
 ('aboard', 1),
 ('absentee', -1),
 ('absentees', -1),
 ('absolve', 2),
 ('absolved', 2),
 ('absolves', 2),
 ('absolving', 2),
 ('absorbed', 1)]

In [ ]:
import spacy
#Custom function :score each word in a sentence in lemmatised form,
#but calculate the score for the whole original sentence.
nlp = spacy.load('en_core_web_sm')
sentiment_lexicon = affinity_scores

def calculate_sentiment(data: str = None) -> float:
    sent_score = 0
    if data:
        sentence = nlp(data)
        for word in sentence:
            sent_score += sentiment_lexicon.get(word.lemma_, 0)
    return sent_score
# test that it works
calculate_sentiment(data = 'very sad')
-2
data_df['sentiment_value'] = data_df['Text'].apply(calculate_sentiment)
data_df

,Text,sentiment_value
0,today is great day,3
1,today is bad day,-3
2,having fun,4
3,its too difficult,-1
4,good morning,3
5,its worth it,2
6,this is not an easy task,1
7,bad day,-3
8,this is kinda boring,-3


In [ ]:
# how many words are in the Text?
data_df['word_count'] = data_df['Text'].str.split().apply(len)
data_df['word_count'].head(10)

0    4
1    4
2    2
3    3
4    2
5    3
6    6
7    2
8    4
Name: word_count, dtype: int64

In [ ]:
data_df.sort_values(by='sentiment_value').tail(10)

,Text,sentiment_value,word_count
1,today is bad day,-3,4
7,bad day,-3,2
8,this is kinda boring,-3,4
3,its too difficult,-1,3
6,this is not an easy task,1,6
5,its worth it,2,3
0,today is great day,3,4
4,good morning,3,2
2,having fun,4,2


In [ ]:
# Sentiment score of the whole review
data_df['sentiment_value'].describe()

count    9.000000
mean     0.333333
std      2.872281
min     -3.000000
25%     -3.000000
50%      1.000000
75%      3.000000
max      4.000000
Name: sentiment_value, dtype: float64

In [ ]:
# Sentiment score of the negative reviews
data_df[data_df['sentiment_value']<0].head(10)

,Text,sentiment_value,word_count
1,today is bad day,-3,4
3,its too difficult,-1,3
7,bad day,-3,2
8,this is kinda boring,-3,4


In [ ]:
# Sentiment score of the positive reviews
data_df[data_df['sentiment_value']>0].head(10)

,Text,sentiment_value,word_count
0,today is great day,3,4
2,having fun,4,2
4,good morning,3,2
5,its worth it,2,3
6,this is not an easy task,1,6


In [ ]:
data_df[data_df['sentiment_value']>10].head(10)

,Text,sentiment_value,word_count


In [ ]:
data_df

,Text,sentiment_value,word_count
0,today is great day,3,4
1,today is bad day,-3,4
2,having fun,4,2
3,its too difficult,-1,3
4,good morning,3,2
5,its worth it,2,3
6,this is not an easy task,1,6
7,bad day,-3,2
8,this is kinda boring,-3,4


In [ ]:
result = []
for value in data_df["sentiment_value"]:
    if value < 0 :
        result.append("Negative")
    elif value == 0 :
        result.append("Neutral")
    else :
        result.append("Positive")

data_df["sentiment_value"] = result
print(data_df)

                       Text sentiment_value  word_count
0        today is great day        Positive           4
1          today is bad day        Negative           4
2                having fun        Positive           2
3         its too difficult        Negative           3
4              good morning        Positive           2
5              its worth it        Positive           3
6  this is not an easy task        Positive           6
7                   bad day        Negative           2
8      this is kinda boring        Negative           4


In [ ]:
data_df

,Text,sentiment_value,word_count
0,today is great day,Positive,4
1,today is bad day,Negative,4
2,having fun,Positive,2
3,its too difficult,Negative,3
4,good morning,Positive,2
5,its worth it,Positive,3
6,this is not an easy task,Positive,6
7,bad day,Negative,2
8,this is kinda boring,Negative,4
